Hussein Hamie
500876254
AER 850 Machine Learning
Assignment 3

# Importing Libraries

In [ ]:
import pandas as pd
import sys
import scipy.io
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn
import numpy as np
import sklearn
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
np.set_printoptions(threshold=sys.maxsize)
tf.config.experimental.enable_tensor_float_32_execution(False)

# Loading in Data

In [ ]:
baselineDE = pd.read_excel('/mnt/d/AER 850 Machine Learning/baseline.xlsx',sheet_name="Baseline DE")

baseline007DE = pd.read_excel('/mnt/d/AER 850 Machine Learning/baseline.xlsx',sheet_name="0.007 DE")

baseline021DE = pd.read_excel('/mnt/d/AER 850 Machine Learning/baseline.xlsx',sheet_name="0.021 DE")

In [ ]:
baselineDE.head()

In [ ]:
baselineDE.insert(0, 'Fault', 0)
baselineDE = baselineDE.rename(columns={'Baseline 0HP DE X97':'0HP','Baseline 1HP DE X98':'1HP','Baseline 2HP DE X99':'2HP','Baseline 3HP DE':'3HP'})
baselineDE.head()

In [ ]:
baseline007DE.insert(0,'Fault',1)
baseline007DE = baseline007DE.rename(columns={'12K 0.007" 0HP':'0HP','12K 0.007" 1HP':'1HP','12K 0.007" 2HP':'2HP','12K 0.007" 3HP':'3HP'})
baseline007DE.head()

In [ ]:
baseline021DE.insert(0,'Fault',1)
baseline021DE = baseline021DE.rename(columns={'12K 0.021" 0HP':'0HP','12K 0.021" 1HP':'1HP','12K 0.021" 2HP':'2HP','12K 0.021" 3HP':'3HP'})
baseline021DE.head()

In [ ]:
faultdata = pd.concat([baselineDE,baseline007DE,baseline021DE])
faultdata.head()

In [ ]:
faultdatashuffled = faultdata.sample(frac=1,random_state = 42)
faultdatashuffled.head()

In [ ]:
def plot_columns(dataframe_cols):
    fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(16, 9), sharey=True)
    colors = plt.cm.get_cmap('Set1', len(axes[0]))
    for i, ax in enumerate(axes.flatten()):
        if i < len(dataframe_cols):
            col = dataframe_cols[i]
            ax.scatter(col.index, col.values, s=0.05, color=colors(i % len(axes[0])))
            ax.set_title(col.name)
        else:
            ax.set_visible(False)
    for i, ax in enumerate(axes[:, 0]):
        if i == 0:
            ax.set_ylabel('normal', rotation=0, fontsize='large', labelpad=40)
        elif i == 1:
            ax.set_ylabel('0.0071 fault', rotation=0, fontsize='large', labelpad=40)
        elif i == 2:
            ax.set_ylabel('0.021 fault', rotation=0, fontsize='large', labelpad=40)
    plt.tight_layout()
    plt.show()

In [ ]:
listsofdatas= [baselineDE['0HP'],baselineDE['1HP'],baselineDE['2HP'],baselineDE['3HP'],baseline007DE['0HP'],baseline007DE['1HP'],baseline007DE['2HP'],baseline007DE['3HP'],baseline021DE['0HP'],baseline021DE['1HP'],baseline021DE['2HP'],baseline021DE['3HP']]
plot_columns(listsofdatas)

In [ ]:
def transpose_and_add_fault_column(df,fault):
    # drop 'Fault' column (if it exists)
    if 'Fault' in df.columns:
        df = df.drop(columns=['Fault'])

    # transpose DataFrame
    df = pd.DataFrame(df.values.T, index=df.columns, columns=df.index)

    # add 'Fault' column with all values set to the fault
    # df.insert(0, 'Fault', fault)

    # return the modified DataFrame
    return df

In [272]:
df = pd.concat([trans_baselineDE, trans_baseline007DE, trans_baseline021DE],axis=0)

In [273]:
mask = df.isna()
first_nan_index = mask.any(axis=0).idxmax()
print(first_nan_index)

121265


In [274]:
data=df.iloc[:,:first_nan_index]
data

,0,1,2,3,4,5,6,7,8,9,...,121255,121256,121257,121258,121259,121260,121261,121262,121263,121264
0HP,0.053197,0.088662,0.099718,0.058621,-0.004590,-0.056952,-0.071764,-0.058621,-0.046521,-0.049859,...,0.027954,-0.024825,-0.088244,-0.135391,-0.147908,-0.109523,-0.025034,0.027954,0.083029,0.128090
1HP,0.145667,0.097796,0.054856,0.036982,0.054445,0.021162,-0.003698,-0.010684,0.029380,0.104576,...,-0.145256,-0.118753,-0.078689,-0.005136,0.041502,0.033900,0.037393,0.057733,0.061431,0.034311
2HP,0.064254,0.063002,-0.004381,-0.035882,-0.023991,0.005215,0.030249,0.007510,-0.016481,-0.041932,...,-0.002295,0.037134,0.042558,0.046104,0.044226,0.068634,0.076353,0.027746,-0.049233,-0.112652
3HP,0.014603,0.054449,0.107646,0.133722,0.112652,0.082403,0.086993,0.110566,0.127673,0.113487,...,-0.002921,-0.051737,-0.066757,-0.038802,0.026077,0.090956,0.081986,0.024408,-0.030458,-0.024825
0HP,-0.083004,-0.195734,0.233419,0.103958,-0.181115,0.055553,0.173806,-0.046944,-0.111918,0.059614,...,0.198333,-0.118578,-0.161136,-0.268505,-0.112568,0.324545,0.142456,-0.316424,-0.063675,0.267368
1HP,-0.277602,-0.044345,0.117603,-0.145055,-0.111430,0.130923,0.032812,-0.197034,-0.074883,0.009584,...,0.048568,-0.294332,-0.048081,0.197196,-0.070172,-0.089014,0.206130,0.042883,-0.279551,0.036223
2HP,-0.093238,0.187288,0.217663,0.070172,0.100385,0.156587,-0.011208,-0.132060,-0.127512,-0.157887,...,-0.019492,-0.051817,0.037198,0.022903,-0.220749,-0.208892,0.125237,0.260871,-0.131735,-0.382372
3HP,0.222699,0.093238,-0.146516,0.177217,0.248526,-0.071147,-0.121339,-0.013969,0.120202,0.071309,...,-0.195897,-0.017381,0.094537,-0.137095,-0.186638,0.158049,0.037523,-0.207105,-0.215064,-0.439062
0HP,1.189431,-0.177866,-0.774816,0.501518,0.993697,-0.348017,-0.811363,0.424362,0.988012,0.089339,...,-0.386190,-0.520605,0.663141,0.909231,-0.319185,-1.363643,-0.231470,0.944966,-0.082030,-1.362831
1HP,0.171369,0.117765,-0.097055,0.009746,0.060913,-0.072690,-0.056040,0.099085,0.138476,-0.008934,...,-0.205887,-0.132385,0.548625,0.335835,-0.205887,0.055634,0.289947,-0.140506,-0.254617,0.198983


Now the NAN's are removed we can break the data into smaller 1000 chunks this way we can do a train test split with more than the 12 tests we have. This is possible because the data is relatively uniform over the course of time in each test


In [287]:
normal = data.iloc[:4]
fault = data.iloc[4:]

In [288]:
def split_dataframe(df, chunk_size,fault):
    """
    Split a pandas DataFrame into smaller DataFrames with a specified number of rows.
    Parameters:
    df (pandas.DataFrame): The DataFrame to split.
    chunk_size (int): The number of rows per smaller DataFrame. Default is 100.
    Returns:
    A list of smaller DataFrames.
    """
    # divide the DataFrame into smaller chunks with chunk_size rows each
    df = df.T
    dfs = np.array_split(df, len(df) // chunk_size)
    print(len(df))
    for i in range(len(dfs)):
        dfs[i] = dfs[i].reset_index(drop=True)
    dfs = pd.concat(dfs, axis=1)

    dfs = dfs.T
    dfs.insert(0,'Fault',fault)

    mask = dfs.isna()
    first_nan_index = mask.any(axis=0).idxmax()
    dfs=dfs.iloc[:,:first_nan_index]


  # for i, chunk in enumerate(df_chunks):
  #       prefix = f"{i}"
  #       new_column_names = {old_col: prefix + str(i) + "_" + old_col for old_col in chunk.columns}
  #       chunk = chunk.rename(columns=new_column_names)
  #       df_chunks[i] = chunk

    return dfs

In [289]:
normal = split_dataframe(normal,2000,0)
fault = split_dataframe(fault,2000,1)

7133
7133


In [290]:
data=pd.concat([normal,fault],axis=0)
data

,Fault,0,1,2,3,4,5,6,7,8,...,2366,2367,2368,2369,2370,2371,2372,2373,2374,2375
0HP,0,0.000000,0.053197,0.088662,0.099718,0.058621,-0.004590,-0.056952,-0.071764,-0.058621,...,0.155836,0.136017,0.096172,0.074684,0.059664,0.076770,0.073433,0.062585,0.046104,0.024199
1HP,0,0.000000,0.145667,0.097796,0.054856,0.036982,0.054445,0.021162,-0.003698,-0.010684,...,-0.029996,0.025887,0.068827,0.099851,0.080949,0.054240,0.031435,0.026709,0.048076,0.006369
2HP,0,0.000000,0.064254,0.063002,-0.004381,-0.035882,-0.023991,0.005215,0.030249,0.007510,...,0.063002,0.027329,0.015020,0.018150,0.022948,0.002503,-0.044644,-0.075102,-0.062793,-0.010222
3HP,0,0.000000,0.014603,0.054449,0.107646,0.133722,0.112652,0.082403,0.086993,0.110566,...,-0.005215,0.073224,0.082612,0.042975,-0.010431,-0.017941,0.012308,0.021279,0.012308,-0.055283
0HP,0,0.029623,-0.005841,-0.044644,-0.044018,-0.038385,-0.017524,-0.004172,-0.000626,0.002295,...,0.007510,-0.007719,0.004381,0.004381,0.010014,0.042349,0.086158,0.135391,0.151038,0.112026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3HP,1,-0.082355,-0.015919,0.431265,0.335916,-0.166658,-0.339489,-0.106882,0.094862,0.202719,...,0.038497,-0.137420,-0.195897,-0.017381,0.094537,-0.137095,-0.186638,0.158049,0.037523,-0.207105
0HP,1,-1.177655,-1.128112,0.617660,0.704968,-0.986387,-1.535824,0.155938,1.055828,-0.402433,...,0.556340,0.795526,-0.386190,-0.520605,0.663141,0.909231,-0.319185,-1.363643,-0.231470,0.944966
1HP,1,-0.061319,0.700501,0.507610,-0.489742,-0.342738,0.834510,0.769943,-0.488118,-0.261927,...,0.325276,0.478371,-0.205887,-0.132385,0.548625,0.335835,-0.205887,0.055634,0.289947,-0.140506
2HP,1,-0.192080,0.376850,-0.088527,-0.354109,0.283043,0.399997,-0.267206,-0.310251,0.068223,...,-0.758978,0.106801,0.906388,0.231876,-0.585985,-0.159999,0.561619,0.277358,-0.382941,-0.311875


In [291]:
X = data[data.columns[1:]].values
y = data[data.columns[0]].values

In [292]:
X.shape,y.shape

((612, 2376), (612,))

In [293]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
len(X_train)


244

In [306]:
 def reshaper(X_train,X_test=0):
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1], 1)
    input_shape = (1, X_train.shape[2], 1)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= np.max(X_train)
    X_test /= np.max(X_train)
    return X_train, X_test, input_shape

In [295]:
X_train,X_validation,input_shape = reshaper(X_train,X_validation)


In [296]:
keras.backend.clear_session()
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(1, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=(1, 2), strides=(1, 2)))
model.add(keras.layers.Conv2D(64, kernel_size=(1, 5), strides=(1, 1), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(1, 2), strides=(1, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [260]:
model.evaluate(X_test, y_test)

ValueError: in user code:

    File "/home/hhamie/miniconda3/envs/tfwsl/lib/python3.10/site-packages/keras/engine/training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "/home/hhamie/miniconda3/envs/tfwsl/lib/python3.10/site-packages/keras/engine/training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/hhamie/miniconda3/envs/tfwsl/lib/python3.10/site-packages/keras/engine/training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "/home/hhamie/miniconda3/envs/tfwsl/lib/python3.10/site-packages/keras/engine/training.py", line 1665, in test_step
        y_pred = self(x, training=False)
    File "/home/hhamie/miniconda3/envs/tfwsl/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/hhamie/miniconda3/envs/tfwsl/lib/python3.10/site-packa

In [297]:
history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_validation, y_validation))

Epoch 1/200
4/4 [==============================] - 1s 43ms/step - loss: 0.3012 - accuracy: 0.9877 - val_loss: 2.8082e-04 - val_accuracy: 1.0000
Epoch 2/200
4/4 [==============================] - 0s 14ms/step - loss: 0.0751 - accuracy: 0.9877 - val_loss: 2.5131e-07 - val_accuracy: 1.0000
Epoch 3/200
4/4 [==============================] - 0s 13ms/step - loss: 0.1090 - accuracy: 0.9877 - val_loss: 3.2544e-07 - val_accuracy: 1.0000
Epoch 4/200
4/4 [==============================] - 0s 14ms/step - loss: 0.0753 - accuracy: 0.9877 - val_loss: 2.5918e-05 - val_accuracy: 1.0000
Epoch 5/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0650 - accuracy: 0.9877 - val_loss: 4.6462e-04 - val_accuracy: 1.0000
Epoch 6/200
4/4 [==============================] - 0s 12ms/step - loss: 0.0645 - accuracy: 0.9877 - val_loss: 2.7450e-04 - val_accuracy: 1.0000
Epoch 7/200
4/4 [==============================] - 0s 12ms/step - loss: 0.0576 - accuracy: 0.9877 - val_loss: 5.8466e-05 - val_accuracy:

In [298]:
score = model.evaluate(X_validation, y_validation, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.7197050894424137e-08
Test accuracy: 1.0


# Running model with 70 30 split

In [299]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
print(len(X_train))
print(len(X_train))

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
len(X_validation)

306
306


92

In [300]:
X_train,X_validation,input_shape = reshaper(X_train,X_validation)


In [301]:
history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_validation, y_validation))

Epoch 1/200
4/4 [==============================] - 0s 35ms/step - loss: 0.0058 - accuracy: 0.9907 - val_loss: 0.0068 - val_accuracy: 0.9891
Epoch 2/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0058 - accuracy: 0.9907 - val_loss: 0.0068 - val_accuracy: 0.9891
Epoch 3/200
4/4 [==============================] - 0s 14ms/step - loss: 0.0059 - accuracy: 0.9907 - val_loss: 0.0068 - val_accuracy: 0.9891
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 0.0058 - accuracy: 0.9907 - val_loss: 0.0068 - val_accuracy: 0.9891
Epoch 5/200
4/4 [==============================] - 0s 16ms/step - loss: 0.0058 - accuracy: 0.9907 - val_loss: 0.0067 - val_accuracy: 0.9891
Epoch 6/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0058 - accuracy: 0.9907 - val_loss: 0.0067 - val_accuracy: 0.9891
Epoch 7/200
4/4 [==============================] - 0s 17ms/step - loss: 0.0058 - accuracy: 0.9907 - val_loss: 0.0067 - val_accuracy: 0.9891
Epoch 8/200
4/4 [===

In [302]:
score = model.evaluate(X_validation, y_validation, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.005710979457944632
Test accuracy: 0.989130437374115


In [329]:
a=np.array(data.iloc[2,1:])
a=a.reshape(1,1,2376,1)
a

array([[[[ 0.00000000e+00],
         [ 6.42535385e-02],
         [ 6.30018462e-02],
         [-4.38092308e-03],
         [-3.58818462e-02],
         [-2.39907692e-02],
         [ 5.21538462e-03],
         [ 3.02492308e-02],
         [ 7.51015385e-03],
         [-1.64806154e-02],
         [-4.19316923e-02],
         [-3.10836923e-02],
         [ 6.67569231e-03],
         [ 2.14873846e-02],
         [ 1.64806154e-02],
         [-6.25846154e-04],
         [ 1.96098462e-02],
         [ 5.67433846e-02],
         [ 1.00344000e-01],
         [ 1.10983385e-01],
         [ 5.11107692e-02],
         [ 6.25846154e-04],
         [-1.06393846e-02],
         [ 1.50203077e-02],
         [ 3.10836923e-02],
         [ 5.21538462e-03],
         [-1.31427692e-02],
         [-3.96369231e-03],
         [ 5.88295385e-02],
         [ 1.27672615e-01],
         [ 1.59799385e-01],
         [ 1.47699692e-01],
         [ 8.07341538e-02],
         [ 5.27796923e-02],
         [ 6.80086154e-02],
         [ 8.5949538

In [330]:
s=model.predict(a)

# Print the predicted class
if s > 0.5:
    print("The new data point is predicted to be faulty.")
else:
    print("The new data point is predicted to be normal.")

1/1 [==============================] - 0s 19ms/step
The new data point is predicted to be faulty.


In [332]:
print(data.iloc[2])

Fault    0.000000
0        0.000000
1        0.064254
2        0.063002
3       -0.004381
           ...   
2371     0.002503
2372    -0.044644
2373    -0.075102
2374    -0.062793
2375    -0.010222
Name: 2HP, Length: 2377, dtype: float64
